In [672]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.tree
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
import pip
pip.main(["install", "openpyxl"])
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [673]:
df = pd.read_excel('/Users/allenclark/Downloads/flight_data/Data_Train.xlsx')

In [674]:
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


In [675]:
# Drop all rows which have null values
df = df.dropna(axis=0)

In [676]:
X = df[df.columns[:-1]]
y = df[df.columns[-1]]

In [677]:
X['Date_of_Journey'] = pd.to_datetime(X['Date_of_Journey'])

/Users/allenclark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [678]:
#function to convert string in duration column to integer of minutes flown
def ret(s):
    s = s.split(' ')
    l = len(s)
    if l > 1:
        return int(s[0].replace('h','')) * 60 + int(s[1].replace('m',''))
    else:
        if 'h' in s[0]:
            return int(s[0].replace('h','')) * 60
        if 'm' in s[0]:
            return int(s[0].replace('m',''))
    
ret('5h 8m')


308

In [679]:
# Get ride of 'Route' and 'Arrival_Time' columns (not much info in route and arrival time equal to departure and duration)
X = X.drop(['Arrival_Time'],axis=1)
#call function to convert duration to integer of minutes flown 
X['Duration'] = X['Duration'].apply(ret)

In [680]:

def convert(df,feature):
    number = preprocessing.LabelEncoder()
    df[feature] = number.fit_transform(df[feature])
  
    return df


In [681]:
for val in ['Airline','Source','Additional_Info','Destination','Total_Stops','Route']:
    X = convert(X,val)

In [682]:
X

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info
0,3,2019-03-24,0,5,18,22:20,170,4,8
1,1,2019-01-05,3,0,84,05:50,445,1,8
2,4,2019-09-06,2,1,118,09:25,1140,1,8
3,3,2019-12-05,3,0,91,18:05,325,0,8
4,3,2019-01-03,0,5,29,16:50,285,0,8
5,8,2019-06-24,3,0,64,09:00,145,4,8
6,4,2019-12-03,0,5,5,18:55,930,0,5
7,4,2019-01-03,0,5,5,08:00,1265,0,8
8,4,2019-12-03,0,5,5,08:55,1530,0,5
9,6,2019-05-27,2,1,104,11:25,470,0,8


In [683]:
X['Date_of_Journey'].dtype

dtype('<M8[ns]')

In [684]:
def weekday(df):
    return df.dayofweek

In [685]:
def month(df):
    return  df.month

In [686]:
def day_of_year(df):
    return df.dayofyear

In [687]:
X.insert(2, 'Month', X['Date_of_Journey'].apply(month))
X.insert(3, 'Day_Of_Week', X['Date_of_Journey'].apply(weekday))
X.insert(1,'day_of_year',X['Date_of_Journey'].apply(day_of_year))

In [688]:
X = X.drop('Date_of_Journey',axis=1)


In [689]:
df_time = pd.to_datetime(X['Dep_Time'])
X['Dep_Time'] = (df_time.dt.hour*60+df_time.dt.minute)*60 + df_time.dt.second

In [690]:
# drop date now that month and day columns have been created
# X = X.drop(['Date_of_Journey'],axis=1)

In [691]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4,random_state=0)

In [692]:
rec = DecisionTreeRegressor(random_state=0,max_depth=8,min_samples_split=4,min_samples_leaf=6)
lin = LinearRegression()


In [693]:
rec.fit(X_train,y_train)
lin.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [694]:
# i_cur = 0
# j_cur = 0
# k_cur = 0
# l = [10000000]
# for i in range(2,20):
#     for j in range(2,20):
#         for k in range(2,20):
#             rec = DecisionTreeRegressor(random_state=0, max_depth=i, min_samples_split=j,min_samples_leaf=k)
#             rec.fit(X_train,y_train)
#             std = (rec.predict(X_test) - y_test).std()
#             if std < min(l):
#                 i_cur = i
#                 j_cur = j
#                 k_url = k
#             l.append(std)
# print('max_depth:', i_cur, 'min_samples:', j_cur)
print(i_cur,j_cur,k_url)
        

19 16 6


In [695]:
forest = RandomForestRegressor(max_depth=15, min_samples_split=5)
forest.fit(X_train,y_train)
(forest.predict(X_test) - y_test).describe()

/Users/allenclark/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


count     4273.000000
mean        23.634978
std       1983.491198
min     -35805.082143
25%       -346.855472
50%         19.413025
75%        487.984445
max      14844.971429
Name: Price, dtype: float64

In [696]:
rec = DecisionTreeRegressor(max_depth=19, min_samples_split=16, min_samples_leaf=6)
rec.fit(X_train,y_train)
(rec.predict(X_test) - y_test).describe()

count     4273.000000
mean        40.215507
std       2220.401138
min     -34997.545455
25%       -407.250000
50%          0.000000
75%        503.000000
max      29579.333333
Name: Price, dtype: float64

In [704]:
boost = GradientBoostingRegressor(n_estimators=2500,min_samples_split=18)
boost.fit(X_train,y_train)
(boost.predict(X_test) - y_test).describe()

count     4273.000000
mean        -0.869204
std       1700.074207
min     -37273.872114
25%       -524.422549
50%         21.827284
75%        561.758363
max      20486.217625
Name: Price, dtype: float64